Импортируем библиотеки

In [32]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.feature_extraction import DictVectorizer as DV

Задаём классификатор

In [33]:
classifier = MLPClassifier(hidden_layer_sizes = 20, max_iter = 1000, solver='adam')

Считываем данные из файла train.csv и выкидываем оттуда столбцы - Name и Ticket, так как они не имеют смысла

In [34]:
X = pd.read_csv('train.csv')
columns_to_save = [X.columns]
y = X['Survived']
X = X.drop('Survived', 1)
X = X.drop('Name', 1)
X = X.drop('Ticket', 1)
X.head()

,PassengerId,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked
0,1,3,male,22.0,1,0,7.2500,NaN,S
1,2,1,female,38.0,1,0,71.2833,C85,C
2,3,3,female,26.0,0,0,7.9250,NaN,S
3,4,1,female,35.0,1,0,53.1000,C123,S
4,5,3,male,35.0,0,0,8.0500,NaN,S


Посмотрим, какими типами являются наши переменные

In [36]:
X.dtypes

PassengerId      int64
Pclass           int64
Sex             object
Age            float64
SibSp            int64
Parch            int64
Fare           float64
Cabin           object
Embarked        object
dtype: object

Выделим отдельно вещественные и категориальные признаки

In [37]:
numeric_cols = ['PassengerId', 'Pclass', 'Age', 'SibSp', 'Parch', 'Fare']
categorical_cols = list(set(X.columns.values.tolist()) - set(numeric_cols))

Создадим функцию для подсчёта средних в каждом столбце

In [38]:
def calculate_means(numeric_data):
    means = np.zeros(numeric_data.shape[1])
    for j in range(numeric_data.shape[1]):
        to_sum = numeric_data.iloc[:,j]
        indices = np.nonzero(~numeric_data.iloc[:,j].isnull())[0]
        correction = np.amax(to_sum[indices])
        to_sum /= correction
        for i in indices:
            means[j] += to_sum[i]
        means[j] /= indices.size
        means[j] *= correction
    return pd.Series(means, numeric_data.columns)

Подсчитаем средние и заменим NaN на средние

In [39]:
X_means = calculate_means(X[numeric_cols])
for i in X[numeric_cols].columns:
    X[numeric_cols] = X[numeric_cols].replace(np.nan, X_means[i])
X.head()

C:\Users\Boss\Anaconda3\lib\site-packages\numpy\core\fromnumeric.py:56: FutureWarning: Series.nonzero() is deprecated and will be removed in a future version.Use Series.to_numpy().nonzero() instead
  return getattr(obj, method)(*args, **kwds)


,PassengerId,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked
0,1,3,male,22.0,1,0,7.2500,NaN,S
1,2,1,female,38.0,1,0,71.2833,C85,C
2,3,3,female,26.0,0,0,7.9250,NaN,S
3,4,1,female,35.0,1,0,53.1000,C123,S
4,5,3,male,35.0,0,0,8.0500,NaN,S


Создадим отдельный массив для хранения категориальных признаков

In [40]:
X_cat = X[categorical_cols]
X_cat.head()

,Sex,Cabin,Embarked
0,male,NaN,S
1,female,C85,C
2,female,NaN,S
3,female,C123,S
4,male,NaN,S


Заменим NaN на строки 'NaN'

In [42]:
X_cat = X_cat.replace(np.nan, 'NaN')

Закодируем категориальные признаки с помощью One Hot Encoding

In [43]:
encoder = DV(sparse = False)
X_cat_oh = encoder.fit_transform(X_cat.T.to_dict().values())

Соберём всё обратно в массив. Теперь все наши данные - вещественные.

In [45]:
X = np.hstack([X[numeric_cols], X_cat_oh])

Взглянем на размерность данных

In [46]:
X.shape

(891, 160)

Разбиваем данные на тестовую и обучающую выборку
Обучаем классификатор на обучающей выборке
Смотрим на результат в переменной score - доля верных ответов

In [48]:
# preprocess dataset, split into training and test part
# X = StandardScaler().fit_transform(X)
X_train, X_test, y_train, y_test = \
    train_test_split(X, y, test_size=.4, random_state=42)

classifier.fit(X_train, y_train)
score = classifier.score(X_test, y_test)

In [50]:
score

0.7450980392156863

Теперь возьмём тестовые данные для kaggle и сделаем всё то же самое

In [20]:
X_test = pd.read_csv('test.csv')
X_train = pd.read_csv('train.csv')
y = X_train['Survived']
X_train = X_train.drop('Survived', 1)

In [21]:
X_test.shape

(418, 11)

In [22]:
X_train.shape

(891, 11)

In [23]:
X = X_train.merge(X_test, how='outer')
X.shape

(1309, 11)

In [24]:
X_means = calculate_means(X[numeric_cols])
for i in X[numeric_cols].columns:
    X[numeric_cols] = X[numeric_cols].replace(np.nan, X_means[i])

C:\Users\Boss\Anaconda3\lib\site-packages\numpy\core\fromnumeric.py:56: FutureWarning: Series.nonzero() is deprecated and will be removed in a future version.Use Series.to_numpy().nonzero() instead
  return getattr(obj, method)(*args, **kwds)


In [25]:
X = X.replace(np.nan, 'NaN')

In [26]:
X_cat = X[categorical_cols]

In [27]:
X_cat_oh = encoder.fit_transform(X_cat.T.to_dict().values())

In [28]:
X = np.hstack([X[numeric_cols], X_cat_oh])
X.shape

(1309, 2435)

In [29]:
# preprocess dataset, split into training and test part
# X = StandardScaler().fit_transform(X)
X_train, X_test, y_train, y_test = \
    train_test_split(X[:891], y, test_size=.4, random_state=42)

classifier.fit(X_train, y_train)
score = classifier.score(X_test, y_test)

In [30]:
prediction = classifier.predict(X[891:])

In [31]:
prediction = pd.DataFrame(data=[X[891:,0], prediction]).T
prediction.columns = ['PassengerId', 'Survived']
prediction = prediction.astype('int')
prediction.head()

,PassengerId,Survived
0,892,0
1,893,1
2,894,0
3,895,0
4,896,1


In [339]:
prediction.to_csv('predict.csv', index=False)